In [1]:
#import tensorflow as tf
import numpy as np
#import keras_cv
import cv2
#from tensorflow import keras
from matplotlib import pyplot as plt
import carla
from enum import Enum
from ultralytics import YOLO
import easyocr
import paho.mqtt.client as mqtt
import pygame

client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
spectator = world.get_spectator()
model_input_shape = 1024
detected_plates = {}


MQTT_ADDRESS = "127.0.0.1"
MQTT_PORT = 1883
#MQTT_TOPIC = "plate/detection"
sender = mqtt.Client()
sender.connect(MQTT_ADDRESS, MQTT_PORT)

class CameraPosition(Enum):
    FRONT = 0
    BACK = 1

def spawn_vehicle(vehicle_index=0, spawn_index=0, pattern='vehicle.*'):
    blueprint_library = world.get_blueprint_library()
    vehicle_bp = blueprint_library.filter(pattern)[vehicle_index]
    spawn_point = world.get_map().get_spawn_points()[spawn_index]
    vehicle = world.spawn_actor(vehicle_bp, spawn_point)
    return vehicle

def resize_image(image):
    if max(image.shape[0], image.shape[1]) > model_input_shape:
        if image.shape[0] > image.shape[1]:
            height = model_input_shape
            width = int(image.shape[1] * model_input_shape / image.shape[0])
        else:
            height = int(image.shape[0] * model_input_shape / image.shape[1])
            width = model_input_shape
    resized_image = cv2.resize(image, (int(width), int(height)), cv2.INTER_AREA)
    resized_image = cv2.copyMakeBorder(resized_image, 0, model_input_shape - resized_image.shape[0], 0, model_input_shape - resized_image.shape[1], cv2.BORDER_CONSTANT, value=[0, 0, 0])
    scale = model_input_shape / max(image.shape[0], image.shape[1])
    return scale, resized_image

def update_detected_plates(plates_list, ticks_threshold=30, n_consecutive_ticks=5):
    new_confirmed_plates = []
    removed_plates = []

    #Update or add plates in the detected list
    for plate in plates_list:
        if plate not in detected_plates:
            detected_plates[plate] = {
                "ticks_since_last_detection": 0,
                "consecutive_detections": 1,
                "is_confirmed": False
            }
        else:
            detected_plates[plate]["ticks_since_last_detection"] = 0
            detected_plates[plate]["consecutive_detections"] += 1

        #Confirm detection after n consecutive ticks that is detected
        if (not detected_plates[plate]["is_confirmed"] and
                detected_plates[plate]["consecutive_detections"] >= n_consecutive_ticks):
            detected_plates[plate]["is_confirmed"] = True
            new_confirmed_plates.append(plate)

    #Update ticks and remove stale plates
    stale_plates = []
    for plate, data in detected_plates.items():
        if plate not in plates_list:
            data["ticks_since_last_detection"] += 1
            data["consecutive_detections"] = 0  # Reset consecutive detections if not seen
        if data["ticks_since_last_detection"] > ticks_threshold:
            stale_plates.append(plate)

    for plate in stale_plates:
        removed_plates.append(plate)
        del detected_plates[plate]

    #Generate message
    message = ""
    if new_confirmed_plates:
        message += f"New plates confirmed: {', '.join(new_confirmed_plates)}\n"
    if removed_plates:
        message += f"Plates not detected anymore: {', '.join(removed_plates)}"

    if message:
        send_mqtt_message("plates/detection", message)

def image_to_pygame(image):
    """
    Converts a CARLA image to a format compatible with pygame (RGB).
    """
    # Convert image to numpy array
    array = np.frombuffer(image.raw_data, dtype=np.uint8)
    array = array.reshape((image.height, image.width, 4))  # BGRA format
    array = array[:, :, :3]  # Keep only RGB
    array = array[:, :, ::-1]  # Convert BGR to RGB
    return array
    
def send_mqtt_message(topic, message):
    sender.publish(topic, message)


camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '1920')
camera_bp.set_attribute('image_size_y', '1080')
camera_bp.set_attribute('fov', '30')
camera_bp.set_attribute('sensor_tick', '0.04')
camera_bp.set_attribute('motion_blur_intensity', '0.0')

camera_driver_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_driver_bp.set_attribute('image_size_x', '1920')
camera_driver_bp.set_attribute('image_size_y', '1080')
camera_driver_bp.set_attribute('fov', '90')
camera_driver_bp.set_attribute('sensor_tick', '0.04')
camera_driver_bp.set_attribute('motion_blur_intensity', '0.0')

spawn_point = carla.Transform()

#yolov8_backbone = keras_cv.models.YOLOV8Backbone.from_preset('yolo_v8_s_backbone_coco',load_weights=True)
yolov8_plates_detector = YOLO("yolo_pytorch/best.pt")

#yolov8_plates_detector = keras_cv.models.YOLOV8Detector(backbone=yolov8_backbone,num_classes=1,bounding_box_format='xywh')
#yolov8_plates_detector.compile(optimizer='adam', classification_loss='binary_crossentropy', box_loss='ciou')
#yolov8_plates_detector.load_weights("model_checkpoints_3.8/checkpoint_epoch_24.weights.h5")
reader = easyocr.Reader(["en"], gpu=True)

pygame 2.6.1 (SDL 2.28.4, Python 3.8.20)
Hello from the pygame community. https://www.pygame.org/contribute.html


C:\Users\Claudio\AppData\Local\Temp\ipykernel_34640\2233056386.py:25: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  sender = mqtt.Client()


In [2]:
topic = "plates/detection"
# Callback function to handle incoming messages
def on_message(client, userdata, msg):
    print(f"Received message: '{msg.payload.decode()}' on topic '{msg.topic}' with QoS {msg.qos}")

# Callback function for connection success
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully to broker")
        client.subscribe(topic)  # Subscribe to the topic after connection
        print(f"Subscribed to topic: {topic}")
    else:
        print(f"Connection failed with code {rc}")

# Create an MQTT client
client = mqtt.Client()

# Assign the callback functions
client.on_connect = on_connect
client.on_message = on_message

# Connect to the MQTT broker
client.connect(MQTT_ADDRESS, MQTT_PORT)

# Start the network loop to process incoming messages
client.loop_start()


Connected successfully to broker


C:\Users\Claudio\AppData\Local\Temp\ipykernel_34640\2018153798.py:16: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

Subscribed to topic: plates/detection


In [7]:
ego_vehicle = spawn_vehicle(vehicle_index=2, spawn_index=7)
static_vehicle = spawn_vehicle(spawn_index=9)
running = True
pygame.init()

front_camera_detection = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=1.7, z=1.0), carla.Rotation(pitch=-5)), attach_to=ego_vehicle)
back_camera_detection = world.spawn_actor(camera_bp, carla.Transform(carla.Location(x=-1.7, z=1.0), carla.Rotation(pitch=-5, yaw=-180)), attach_to=ego_vehicle)
front_camera_driver = world.spawn_actor(camera_driver_bp, carla.Transform(carla.Location(x=1.2, z=1.2), carla.Rotation(pitch=-10)), attach_to=ego_vehicle)

video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)
back_video_output = np.zeros((model_input_shape, model_input_shape, 4), dtype=np.uint8)

saved_location = carla.Location(x=0, y=0, z=0)

#Used to enhance the contrast of the read plate, improving visibility and potentially the chance to be correctly read
sharpen_kernel = np.array([[0, -1, 0],
                           [-1, 5, -1],
                           [0, -1, 0]])

def detector_callback(image, camera_position):
    global video_output
    global back_video_output
    image_array = np.frombuffer(image.raw_data, dtype=np.uint8).reshape((image.height, image.width, 4))
    image_bgr = cv2.cvtColor(image_array, cv2.COLOR_BGRA2BGR)
    current_detected_plates = []
    scale, resized_image = resize_image(image_bgr)
    pred = yolov8_plates_detector.predict(resized_image, verbose=False)
    rescaling_factor = 1 / scale
    for result in pred:
        boxes = result.boxes
        if boxes:
            boxes_to_skip = filter_boxes(boxes)
            #print(boxes_to_remove)
            for i in range(len(boxes)):
                if i not in boxes_to_skip:
                    box = boxes[i]
                    plate = image_bgr[int(box.xyxy[0][1].item() * rescaling_factor) : int(box.xyxy[0][3].item() * rescaling_factor), int(box.xyxy[0][0].item() * rescaling_factor) : int(box.xyxy[0][2].item() * rescaling_factor)]
                    plate_enhanced = cv2.filter2D(plate, -1, sharpen_kernel)
                    plate_reading = reader.readtext(plate_enhanced)
                    cv2.rectangle(image_bgr, (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), (int((box.xyxy[0][2].item() * rescaling_factor)), int((box.xyxy[0][3].item() * rescaling_factor))), (0, 255, 0), 1)
                    read_flag = False
                    for plate_detected in plate_reading:
                        confidence = plate_detected[2]
                        plate_text = plate_detected[1]
                        #print(plate_text)
                        if confidence > 0.5:
                            read_flag = True
                            cv2.putText(image_bgr, fix_plate_text(plate_text), (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
                            current_detected_plates.append(plate_text.upper())
                    if not read_flag:
                        cv2.putText(image_bgr, "????", (int(box.xyxy[0][0].item() * rescaling_factor), int(box.xyxy[0][1].item() * rescaling_factor)), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
    update_detected_plates(current_detected_plates)
    if camera_position == CameraPosition.FRONT:
        video_output = image_bgr
    elif camera_position == CameraPosition.BACK:
        back_video_output = image_bgr

def camera_callback(image):
    global driver_sight_scene
    #driver_sight_scene = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    driver_sight_scene = image_to_pygame(image)

def fix_plate_text(plate_text):
    # Undetectable typos: 0-O-D, 1-I, 2-Z, 5-S
    # Detectable only if close enough: 8-B, 1-J
    # Frequent: 4-A
    new_text = plate_text
    new_text = new_text.replace(" ", "")
    new_text = new_text.replace("]", "I")
    new_text = new_text.replace("[", "I")
    new_text = new_text.replace("\"", "")
    new_text = new_text.replace("'", "")
    return new_text


def filter_boxes(boxes):
    boxes_to_remove = []
    if len(boxes) > 1:
        for i in range(len(boxes)):
            for j in range(len(boxes)):
                if i < j:
                    if intersection_over_union(boxes[i].xyxy[0], boxes[j].xyxy[0]) > 0.6:
                        boxes_to_remove.append(j)
    return boxes_to_remove


def intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    unionArea = float(boxAArea + boxBArea - interArea)
    iou = interArea / unionArea
    return iou

front_camera_detection.listen(lambda image: detector_callback(image, CameraPosition.FRONT))
back_camera_detection.listen(lambda image: detector_callback(image, CameraPosition.BACK))
front_camera_driver.listen(lambda image: camera_callback(image))


ego_vehicle_controls = carla.VehicleControl()
ego_vehicle_controls.throttle = 0.2
#ego_vehicle.apply_control(ego_vehicle_controls)
#ego_vehicle.set_autopilot(True)
#static_vehicle.set_autopilot(True)
#print(static_vehicle.attributes.items)

cv2.namedWindow('RGB Front Camera', cv2.WINDOW_AUTOSIZE)
cv2.namedWindow('RGB Back Camera', cv2.WINDOW_AUTOSIZE)
#print(ego_vehicle.attributes)
driver_camera = pygame.display.set_mode((1920, 1080))

try:
    while running:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_w:
                    ego_vehicle_controls.throttle = 0.2
                    ego_vehicle_controls.brake = 0
                elif event.key == pygame.K_s:
                    ego_vehicle_controls.throttle = 0
                    ego_vehicle_controls.brake = 0.2
                elif event.key == pygame.K_q:
                    running = False
                    break

                ego_vehicle.apply_control(ego_vehicle_controls)
        if cv2.waitKey(1) == ord('q'):
            running = False
            break
        if video_output is not None:
            cv2.imshow('RGB Front Camera', video_output)
        if back_video_output is not None:
            cv2.imshow('RGB Back Camera', back_video_output)
        if driver_sight_scene is not None:
            pygame_image = pygame.surfarray.make_surface(driver_sight_scene.swapaxes(0, 1))  # Swap axes for pygame
            driver_camera.blit(pygame_image, (0, 0))
        pygame.display.flip()
        if cv2.waitKey(1) == ord('s'):
            saved_location = ego_vehicle.get_location()
            print("Saved location: ", saved_location)
except Exception as e:
    print(f"Error occurred: {e}")
finally:
    print("Shutting down...")
    cv2.destroyAllWindows()
    if 'front_camera_detection' in locals() and front_camera_detection.is_alive:
        front_camera_detection.destroy()
    if 'front_camera_driver' in locals() and front_camera_driver.is_alive:
        front_camera_driver.destroy()
    if 'back_camera_detection' in locals() and back_camera_detection.is_alive:
        back_camera_detection.destroy()

    for actor in world.get_actors():
        if actor.type_id.startswith('vehicle.') or actor.type_id.startswith('sensor.camera'):
            actor.destroy()
    client.loop_stop()
    pygame.quit()


Shutting down...


In [ ]:
sender.disconnect()

In [3]:
for vehicle in world.get_actors().filter('vehicle.*'):
    vehicle.destroy()


In [50]:
world.set_weather(carla.WeatherParameters.ClearNight)

In [10]:
import torch, torchvision
print(torch.cuda.is_available())
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torchvision.__version__)
